<a href="https://colab.research.google.com/github/DhanviSompalli15/bank-fraud-detection/blob/main/bankfraud_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

sample_data


In [ ]:
# Install required packages
!pip install numpy pandas scikit-learn -q

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# Generate synthetic dataset
np.random.seed(42)
def generate_synthetic_data(n_samples=10000):
    data = {
        'transaction_id': range(1, n_samples + 1),
        'amount': np.random.uniform(10, 10000, n_samples),
        'transaction_time': np.random.uniform(0, 24, n_samples),
        'customer_age': np.random.randint(18, 80, n_samples),
        'account_balance': np.random.uniform(0, 50000, n_samples),
        'num_transactions_last_hour': np.random.randint(1, 10, n_samples),
        'is_online': np.random.choice([0, 1], n_samples, p=[0.4, 0.6]),
        'is_fraud': np.random.choice([0, 1], n_samples, p=[0.98, 0.02])
    }
    df = pd.DataFrame(data)
    fraud_idx = df['is_fraud'] == 1
    df.loc[fraud_idx, 'amount'] *= np.random.uniform(1.5, 3, fraud_idx.sum())
    df.loc[fraud_idx, 'num_transactions_last_hour'] += np.random.randint(2, 5, fraud_idx.sum())
    df.loc[fraud_idx, 'is_online'] = 1
    return df

# Create and train model
df = generate_synthetic_data()
X = df[['amount', 'transaction_time', 'customer_age', 'account_balance',
        'num_transactions_last_hour', 'is_online']]
y = df['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save the model
with open('fraud_detection_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("Model saved as fraud_detection_model.pkl")

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2944
           1       1.00      0.62      0.77        56

    accuracy                           0.99      3000
   macro avg       1.00      0.81      0.88      3000
weighted avg       0.99      0.99      0.99      3000

Confusion Matrix:
[[2944    0]
 [  21   35]]
Model saved as fraud_detection_model.pkl


In [ ]:
import pickle
import numpy as np

# Load the model
with open('fraud_detection_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Function to get prediction
def predict_fraud(amount, transaction_time, customer_age, account_balance,
                 num_transactions, is_online):
    input_data = np.array([[amount, transaction_time, customer_age, account_balance,
                          num_transactions, is_online]])
    prediction = model.predict(input_data)[0]
    probability = model.predict_proba(input_data)[0][1]  # Probability of fraud
    return "Fraudulent" if prediction == 1 else "Legitimate", probability

# Test with your own values
amount = float(input("Enter Transaction Amount ($): "))
transaction_time = float(input("Enter Transaction Time (hours, 0-24): "))
customer_age = int(input("Enter Customer Age: "))
account_balance = float(input("Enter Account Balance ($): "))
num_transactions = int(input("Enter Transactions in Last Hour: "))
is_online = int(input("Is Online Transaction? (0 for No, 1 for Yes): "))

result, prob = predict_fraud(amount, transaction_time, customer_age,
                            account_balance, num_transactions, is_online)
print(f"Prediction: {result} (Fraud Probability: {prob:.2%})")

Enter Transaction Amount ($): 1000
Enter Transaction Time (hours, 0-24): 10
Enter Customer Age: 30
Enter Account Balance ($): 500
Enter Transactions in Last Hour: 5
Is Online Transaction? (0 for No, 1 for Yes): 1
Prediction: Legitimate (Fraud Probability: 2.00%)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
